In [1]:
import configparser
import os.path as op

CP = configparser.ConfigParser()
CP.read_file(open(op.join(op.expanduser('~'), '.aws', 'credentials')))
CP.sections()
aws_access_key = CP.get('hcp', 'AWS_ACCESS_KEY_ID')
aws_secret_key = CP.get('hcp', 'AWS_SECRET_ACCESS_KEY')

def attach_keys(arr):
    return [(e, aws_access_key, aws_secret_key) for e in arr]

In [2]:
def afq_hcp_api(args):
    import logging
    import os.path as op
    import s3fs

    from AFQ.data import fetch_hcp
    import AFQ.api as api
    
    logging.basicConfig(level=logging.INFO)
    log = logging.getLogger(__name__)  
    
    fs = s3fs.S3FileSystem()
    
    subject = args[0]
    aws_access_key = args[1]
    aws_secret_key = args[2]

    fetch_hcp([subject], profile_name=False,
              aws_access_key_id=aws_access_key,
              aws_secret_access_key=aws_secret_key)
    derivatives = op.join(op.expanduser('~'), "AFQ_data/HCP/derivatives");
    
    myafq = api.AFQ(op.join(derivatives, "dmriprep"),
                    b0_threshold=50,
                    sub_prefix="sub-",
                    tracking_params = {"odf_model": "DKI"},
                    scalars=["dti_fa", "dti_md", "dki_fa", "dki_md"])
                        
    myafq.export_bundles()
    myafq.get_tract_profiles()
    myafq.export_bundle_gif()
    myafq.export_ROI_gifs()
    df = myafq.combine_profiles()
    fs.put(op.join(derivatives, "afq"), "profile-hcp", recursive=True)
    return df

In [3]:
import cloudknot as ck
ck.set_region('us-east-1')

In [4]:
knot = ck.Knot(name='hcp_api-64gb-20428-2',
              func=afq_hcp_api,
              base_image='libgl:1',
              image_github_installs="https://github.com/36000/pyAFQ.git@xvfb-viz", # @xvfb-viz
              pars_policies=('AmazonS3FullAccess',),
              resource_type="SPOT",
              bid_percentage=100,
              memory=64000)

In [5]:
result_futures = knot.map(attach_keys(["100206", "162228", "175540"]))

In [12]:
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
90ab5e62-9ae7-45b3-be82-c291880b216b        hcp-api-64gb-20428-2-0        PENDING  


In [9]:
knot.clobber(clobber_pars=True, clobber_repo=True, clobber_image=True)